In [1]:
import findspark
findspark.init("D:/spark")

In [2]:
from pyspark.sql import SparkSession

# SparkSession

In [3]:
spark = SparkSession.builder \
.master("local[4]") \
.appName("ClassificationWithIris") \
.config("spark.driver.memory","2g") \
.config("spark.executor.memory","4g") \
.getOrCreate()

# Veri Setini Okuma

In [6]:
df = spark.read.format("csv") \
.option("header",True) \
.option("sep",",") \
.option("inferSchema", True) \
.load("D:/Datasets/iris.csv")

# 1. Veri Keşfi

In [7]:
df.limit(5).toPandas().head()

c:\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [9]:
df.describe().toPandas().head()

,summary,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,count,150,150,150,150,150
1,mean,5.843333333333335,3.0540000000000007,3.7586666666666693,1.1986666666666672,None
2,stddev,0.8280661279778637,0.43359431136217375,1.764420419952262,0.7631607417008414,None
3,min,4.3,2.0,1.0,0.1,Iris-setosa
4,max,7.9,4.4,6.9,2.5,Iris-virginica


In [10]:
import pyspark.sql.functions as f

In [13]:
df.groupBy("Species").agg(f.count("SepalLengthCm").alias("sayi")).show()

+---------------+----+
|        Species|sayi|
+---------------+----+
| Iris-virginica|  50|
|    Iris-setosa|  50|
|Iris-versicolor|  50|
+---------------+----+



In [14]:
df.printSchema()

root
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



# 2. Veri Temizliği ve Ön Hazırlığı

In [15]:
# kategorik nitelik olmadığı için StringIndexer ve OneHotEncoder yapmıyoruz.
# Boş değer olmadığı için Imputer yapmıyoruz
# Sadece hedef değişken kategorik olduğu için StringIndexer kullanacağız

In [16]:
df.columns

['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']

# 2.1. StringIndexer Aşaması

In [17]:
from pyspark.ml.feature import StringIndexer

In [18]:
indexer = StringIndexer() \
.setHandleInvalid("skip") \
.setInputCol("Species") \
.setOutputCol("label")

In [19]:
indexerDF = indexer.fit(df).transform(df)

In [20]:
indexerDF.toPandas().head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,label
0,5.1,3.5,1.4,0.2,Iris-setosa,0.0
1,4.9,3.0,1.4,0.2,Iris-setosa,0.0
2,4.7,3.2,1.3,0.2,Iris-setosa,0.0
3,4.6,3.1,1.5,0.2,Iris-setosa,0.0
4,5.0,3.6,1.4,0.2,Iris-setosa,0.0


# 2.2. VectorAssembler Aşaması

In [21]:
from pyspark.ml.feature import VectorAssembler

In [23]:
assembler = VectorAssembler() \
.setInputCols(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']) \
.setOutputCol("features")

In [24]:
assembler_df = assembler.transform(indexerDF)

In [25]:
assembler_df.toPandas().head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,label,features
0,5.1,3.5,1.4,0.2,Iris-setosa,0.0,"[5.1, 3.5, 1.4, 0.2]"
1,4.9,3.0,1.4,0.2,Iris-setosa,0.0,"[4.9, 3.0, 1.4, 0.2]"
2,4.7,3.2,1.3,0.2,Iris-setosa,0.0,"[4.7, 3.2, 1.3, 0.2]"
3,4.6,3.1,1.5,0.2,Iris-setosa,0.0,"[4.6, 3.1, 1.5, 0.2]"
4,5.0,3.6,1.4,0.2,Iris-setosa,0.0,"[5.0, 3.6, 1.4, 0.2]"


# 2.3. Standardizasyon Aşaması

In [27]:
# Nitelikler aynı ölçekte olduğu için gerek yok

# 2.4. Veri Setini train ve test olarak bölme

In [28]:
train_df, test_df = assembler_df.randomSplit([0.8, 0.2], seed=142)

In [29]:
train_df.count()

117

In [30]:
test_df.count()

33

# 3. Model Oluşturma

In [31]:
from pyspark.ml.classification import LogisticRegression

In [33]:
# sınıflandırıcı nesnesi oluşturma
logreg_obj = LogisticRegression() \
.setLabelCol("label") \
.setFeaturesCol("features")

In [34]:
# Modeli eğitme
logreg_model = logreg_obj.fit(train_df)

In [35]:
# modeli kullanarak tahmin yapma
transformed_df = logreg_model.transform(test_df)

In [36]:
transformed_df.limit(5).toPandas().head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,label,features,rawPrediction,probability,prediction
0,4.4,3.0,1.3,0.2,Iris-setosa,0.0,"[4.4, 3.0, 1.3, 0.2]","[14604.377893842138, -2761.3173438072095, -118...","[1.0, 0.0, 0.0]",0.0
1,4.6,3.1,1.5,0.2,Iris-setosa,0.0,"[4.6, 3.1, 1.5, 0.2]","[14632.728901273445, -2839.341524260714, -1179...","[1.0, 0.0, 0.0]",0.0
2,4.6,3.6,1.0,0.2,Iris-setosa,0.0,"[4.6, 3.6, 1.0, 0.2]","[20348.465372786737, -5208.036002792378, -1514...","[1.0, 0.0, 0.0]",0.0
3,4.7,3.2,1.3,0.2,Iris-setosa,0.0,"[4.7, 3.2, 1.3, 0.2]","[15819.80674976183, -3254.5791800061547, -1256...","[1.0, 0.0, 0.0]",0.0
4,4.9,2.4,3.3,1.0,Iris-versicolor,1.0,"[4.9, 2.4, 3.3, 1.0]","[-1046.3786273186568, 2375.2529407912994, -132...","[0.0, 1.0, 0.0]",1.0


# 4. Model Değerlendirme

In [37]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [38]:
evaluator = MulticlassClassificationEvaluator() \
.setLabelCol("label") \
.setPredictionCol("prediction") \
.setMetricName("accuracy")

In [39]:
accuracy = evaluator.evaluate(transformed_df)

In [40]:
accuracy

0.9393939393939394